# Haritha Thipparapu 
# HDS 5230 se- High Performance Computing# 
Week 5 Dask Programming Assignment

# Haritha Thipparapu
# HDS 5230 section 07 - High Performance Computing
# Weej 

In [77]:
import numpy as np
import pandas as pd
import cProfile

In [78]:
#%% read in the data
df = pd.read_csv("timeseries.csv")
print(df.head())

C:\Users\harit\AppData\Local\Temp\ipykernel_25292\593144534.py:2: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("timeseries.csv")


                         name   level city   county     state  country  \
0  Antwerp, Flanders, Belgium  county  NaN  Antwerp  Flanders  Belgium   
1  Antwerp, Flanders, Belgium  county  NaN  Antwerp  Flanders  Belgium   
2  Antwerp, Flanders, Belgium  county  NaN  Antwerp  Flanders  Belgium   
3  Antwerp, Flanders, Belgium  county  NaN  Antwerp  Flanders  Belgium   
4  Antwerp, Flanders, Belgium  county  NaN  Antwerp  Flanders  Belgium   

   population      lat    long                          url  ... recovered  \
0   1847486.0  51.2485  4.7175  https://epistat.wiv-isp.be/  ...       NaN   
1   1847486.0  51.2485  4.7175  https://epistat.wiv-isp.be/  ...       NaN   
2   1847486.0  51.2485  4.7175  https://epistat.wiv-isp.be/  ...       NaN   
3   1847486.0  51.2485  4.7175  https://epistat.wiv-isp.be/  ...       NaN   
4   1847486.0  51.2485  4.7175  https://epistat.wiv-isp.be/  ...       NaN   

  active  tested  hospitalized  hospitalized_current  discharged  icu  \
0    NaN     

In [79]:
df.dtypes

name                     object
level                    object
city                     object
county                   object
state                    object
country                  object
population              float64
lat                     float64
long                    float64
url                      object
aggregate                object
tz                       object
cases                   float64
deaths                  float64
recovered               float64
active                  float64
tested                  float64
hospitalized            float64
hospitalized_current    float64
discharged              float64
icu                     float64
icu_current             float64
growthFactor            float64
date                     object
dtype: object

In [80]:
import dask.dataframe as dd

# Define the data types
dtypes = {
    'aggregate': 'object',
    'cases': 'float64',
    'city': 'object',
    'population': 'float64',
    'deaths': 'float64',
    'country': 'object',
    'level': 'object',
    'state': 'object',
    'date': 'object'  # Will be converted to datetime later
}

# Read the CSV file using Dask
CDS = dd.read_csv('timeseries.csv', dtype=dtypes, parse_dates=['date'])

# Convert necessary columns to float
CDS['population'] = CDS['population'].astype(float)
CDS['cases'] = CDS['cases'].astype(float)
CDS['deaths'] = CDS['deaths'].astype(float)

# Print schema to verify
print(CDS.dtypes)


name                    string[pyarrow]
level                   string[pyarrow]
city                    string[pyarrow]
county                  string[pyarrow]
state                   string[pyarrow]
country                 string[pyarrow]
population                      float64
lat                             float64
long                            float64
url                     string[pyarrow]
aggregate               string[pyarrow]
tz                      string[pyarrow]
cases                           float64
deaths                          float64
recovered                       float64
active                          float64
tested                          float64
hospitalized                    float64
hospitalized_current            float64
discharged                      float64
icu                             float64
icu_current                     float64
growthFactor                    float64
date                    string[pyarrow]
dtype: object


In [81]:
import dask.dataframe as dd

# Define column data types to prevent dtype inference issues
dtypes = {
    'name': 'object',
    'level': 'object',
    'city': 'object',
    'county': 'object',
    'state': 'object',
    'country': 'object',
    'population': 'float64',
    'lat': 'float64',
    'long': 'float64',
    'url': 'object',
    'aggregate': 'object',
    'tz': 'object',
    'cases': 'float64',
    'deaths': 'float64',
    'recovered': 'float64',
    'active': 'float64',
    'tested': 'float64',
    'hospitalized': 'float64',
    'hospitalized_current': 'float64',
    'discharged': 'float64',
    'icu': 'float64',
    'icu_current': 'float64',
    'growthFactor': 'float64',
    'date': 'object'
}


# Filter the dataset for US states
df_us = df[df['country'] == 'US']

# Convert date column to datetime format
df_us['date'] = dd.to_datetime(df_us['date'])

# Filter for the period 2020-Jan-01 to 2021-Feb-28
df_us = df_us[(df_us['date'] >= '2020-01-01') & (df_us['date'] <= '2021-02-28')]

# Check the filtered data
df_us.head()

,name,level,city,county,state,country,population,lat,long,url,...,recovered,active,tested,hospitalized,hospitalized_current,discharged,icu,icu_current,growthFactor,date


In [82]:
# Calculate per-capita mortality
df_us['deaths'] = df_us['deaths'].fillna(0)
df_us['population'] = df_us['population'].fillna(0)
df_us['deaths_per_capita'] = df_us['deaths'] / df_us['population']

# Group by state and calculate total deaths and average population
df_state_per_capita = df_us.groupby('state').agg({'deaths': 'sum', 'population': 'mean'}).compute()

# Calculate per-capita mortality and rank states
df_state_per_capita['per_capita_mortality'] = df_state_per_capita['deaths'] / df_state_per_capita['population']
df_state_per_capita = df_state_per_capita.sort_values(by='per_capita_mortality', ascending=False)

# Output the per-capita mortality ranking
print("Per-capita mortality ranking:")
print(df_state_per_capita[['per_capita_mortality']])


AttributeError: 'DataFrame' object has no attribute 'compute'

In [84]:
# Compute Case Fatality Rate (CFR) per month using WHO methodology
df_us['cfr'] = df_us['deaths'] / df_us['cases']

# Extract year and month for CFR calculation
df_us['year_month'] = df_us['date'].dt.to_period('M')

# Compute CFR per state per month
df_cfr = df_us.groupby(['state', 'year_month']).agg({'cfr': 'mean'}).compute()

# Output the CFR matrix (states x months)
print("\nCFR per state per month:")
print(df_cfr.head())

AttributeError: 'DataFrame' object has no attribute 'compute'

In [ ]:
# Reshape data to 50 (states) x 14 (months) matrix
df_cfr_matrix = df_cfr.unstack(fill_value=0).T

# Rank states by changes in CFR (month-to-month)
df_cfr_matrix_diff = df_cfr_matrix.diff(axis=1).fillna(0)
df_cfr_rank = df_cfr_matrix_diff.sum(axis=1).sort_values(ascending=False)

# Output the ranking based on CFR changes
print("\nCFR ranking by month-to-month change:")
print(df_cfr_rank)